In [1]:
from pyspark.sql import SparkSession

# Create a Spark session
spark = SparkSession.builder \
    .appName("olistData") \
    .getOrCreate()



25/03/30 13:26:58 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark

In [3]:
!hadoop fs -ls /data/olist/

Found 9 items
-rw-r--r--   2 abhishekpandey68461 hadoop    9033957 2025-03-30 13:16 /data/olist/olist_customers_dataset.csv
-rw-r--r--   2 abhishekpandey68461 hadoop   61273883 2025-03-30 13:16 /data/olist/olist_geolocation_dataset.csv
-rw-r--r--   2 abhishekpandey68461 hadoop   15438671 2025-03-30 13:16 /data/olist/olist_order_items_dataset.csv
-rw-r--r--   2 abhishekpandey68461 hadoop    5777138 2025-03-30 13:16 /data/olist/olist_order_payments_dataset.csv
-rw-r--r--   2 abhishekpandey68461 hadoop   14451670 2025-03-30 13:16 /data/olist/olist_order_reviews_dataset.csv
-rw-r--r--   2 abhishekpandey68461 hadoop   17654914 2025-03-30 13:16 /data/olist/olist_orders_dataset.csv
-rw-r--r--   2 abhishekpandey68461 hadoop    2379446 2025-03-30 13:16 /data/olist/olist_products_dataset.csv
-rw-r--r--   2 abhishekpandey68461 hadoop     174703 2025-03-30 13:16 /data/olist/olist_sellers_dataset.csv
-rw-r--r--   2 abhishekpandey68461 hadoop       2613 2025-03-30 13:16 /data/olist/product_category_

In [6]:
hdfs_path='/data/olist'

In [7]:
customers_df = spark.read.csv(hdfs_path + "/olist_customers_dataset.csv", header=True, inferSchema=True)
geolocation_df = spark.read.csv(hdfs_path + "/olist_geolocation_dataset.csv", header=True, inferSchema=True)
order_items_df = spark.read.csv(hdfs_path + "/olist_order_items_dataset.csv", header=True, inferSchema=True)
payments_df = spark.read.csv(hdfs_path + "/olist_order_payments_dataset.csv", header=True, inferSchema=True)
reviews_df = spark.read.csv(hdfs_path + "/olist_order_reviews_dataset.csv", header=True, inferSchema=True)
orders_df = spark.read.csv(hdfs_path + "/olist_orders_dataset.csv", header=True, inferSchema=True)
products_df = spark.read.csv(hdfs_path + "/olist_products_dataset.csv", header=True, inferSchema=True)
sellers_df = spark.read.csv(hdfs_path + "/olist_sellers_dataset.csv", header=True, inferSchema=True)
category_translation_df = spark.read.csv(hdfs_path + "/product_category_name_translation.csv", header=True, inferSchema=True)

In [9]:
# Verifying by printing schemas
customers_df.printSchema()
geolocation_df.printSchema()
order_items_df.printSchema()
payments_df.printSchema()
reviews_df.printSchema()
orders_df.printSchema()
products_df.printSchema()
sellers_df.printSchema()
category_translation_df.printSchema()


root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable 

In [11]:
# Count the number of rows in each DataFrame
print("Customers:", customers_df.count())
print("Geolocation:", geolocation_df.count())
print("Order Items:", order_items_df.count())
print("Payments:", payments_df.count())
print("Reviews:", reviews_df.count())
print("Orders:", orders_df.count())
print("Products:", products_df.count())
print("Sellers:", sellers_df.count())
print("Category Translation:", category_translation_df.count())


Customers: 99441


Geolocation: 1000163
Order Items: 112650
Payments: 103886
Reviews: 104162
Orders: 99441
Products: 32951
Sellers: 3095
Category Translation: 71


In [12]:
# Print column names of each DataFrame
print("Customers Columns:", customers_df.columns)
print("Geolocation Columns:", geolocation_df.columns)
print("Order Items Columns:", order_items_df.columns)
print("Payments Columns:", payments_df.columns)
print("Reviews Columns:", reviews_df.columns)
print("Orders Columns:", orders_df.columns)
print("Products Columns:", products_df.columns)
print("Sellers Columns:", sellers_df.columns)
print("Category Translation Columns:", category_translation_df.columns)


Customers Columns: ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
Geolocation Columns: ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state']
Order Items Columns: ['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value']
Payments Columns: ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value']
Reviews Columns: ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']
Orders Columns: ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date']
Products Columns: ['product_id', 'product_category_name', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_we

In [13]:
from pyspark.sql.functions import col, count, when

# Function to check null values in each column
def check_nulls(df, df_name):
    print(f"Null values in {df_name}:")
    df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

# Function to check duplicate rows
def check_duplicates(df, df_name):
    print(f"Duplicate rows in {df_name}: {df.count() - df.dropDuplicates().count()}")

# Checking null and duplicate values for all DataFrames
datasets = {
    "customers_df": customers_df,
    "geolocation_df": geolocation_df,
    "order_items_df": order_items_df,
    "payments_df": payments_df,
    "reviews_df": reviews_df,
    "orders_df": orders_df,
    "products_df": products_df,
    "sellers_df": sellers_df,
    "category_translation_df": category_translation_df
}

for name, df in datasets.items():
    check_nulls(df, name)
    check_duplicates(df, name)


Null values in customers_df:


+-----------+------------------+------------------------+-------------+--------------+
|customer_id|customer_unique_id|customer_zip_code_prefix|customer_city|customer_state|
+-----------+------------------+------------------------+-------------+--------------+
|          0|                 0|                       0|            0|             0|
+-----------+------------------+------------------------+-------------+--------------+



Duplicate rows in customers_df: 0
Null values in geolocation_df:


+---------------------------+---------------+---------------+----------------+-----------------+
|geolocation_zip_code_prefix|geolocation_lat|geolocation_lng|geolocation_city|geolocation_state|
+---------------------------+---------------+---------------+----------------+-----------------+
|                          0|              0|              0|               0|                0|
+---------------------------+---------------+---------------+----------------+-----------------+



Duplicate rows in geolocation_df: 261831
Null values in order_items_df:


...

Duplicate rows in reviews_df: 85
Null values in orders_df:


+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|order_id|customer_id|order_status|order_purchase_timestamp|order_approved_at|order_delivered_carrier_date|order_delivered_customer_date|order_estimated_delivery_date|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+
|       0|          0|           0|                       0|              160|                        1783|                         2965|                            0|
+--------+-----------+------------+------------------------+-----------------+----------------------------+-----------------------------+-----------------------------+



Duplicate rows in orders_df: 0
Null values in products_df:


+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|product_id|product_category_name|product_name_lenght|product_description_lenght|product_photos_qty|product_weight_g|product_length_cm|product_height_cm|product_width_cm|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+
|         0|                  610|                610|                       610|               610|               2|                2|                2|               2|
+----------+---------------------+-------------------+--------------------------+------------------+----------------+-----------------+-----------------+----------------+



Duplicate rows in products_df: 0
Null values in sellers_df:
+---------+----------------------+-----------+------------+
|seller_id|seller_zip_code_prefix|seller_city|seller_state|
+---------+----------------------+-----------+------------+
|        0|                     0|          0|           0|
+---------+----------------------+-----------+------------+

Duplicate rows in sellers_df: 0
Null values in category_translation_df:
+---------------------+-----------------------------+
|product_category_name|product_category_name_english|
+---------------------+-----------------------------+
|                    0|                            0|
+---------------------+-----------------------------+

Duplicate rows in category_translation_df: 0


In [14]:
from pyspark.sql.functions import col

customers_df.groupBy("customer_state").count().orderBy(col("count").desc()).show()


+--------------+-----+
|customer_state|count|
+--------------+-----+
|            SP|41746|
|            RJ|12852|
|            MG|11635|
|            RS| 5466|
|            PR| 5045|
|            SC| 3637|
|            BA| 3380|
|            DF| 2140|
|            ES| 2033|
|            GO| 2020|
|            PE| 1652|
|            CE| 1336|
|            PA|  975|
|            MT|  907|
|            MA|  747|
|            MS|  715|
|            PB|  536|
|            PI|  495|
|            RN|  485|
|            AL|  413|
+--------------+-----+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import col

orders_df.groupBy("order_status").count().orderBy(col("count").desc()).show()


+------------+-----+
|order_status|count|
+------------+-----+
|   delivered|96478|
|     shipped| 1107|
|    canceled|  625|
| unavailable|  609|
|    invoiced|  314|
|  processing|  301|
|     created|    5|
|    approved|    2|
+------------+-----+



In [16]:
from pyspark.sql.functions import col

payments_df.groupBy("payment_type").count().orderBy(col("count").desc()).show()


+------------+-----+
|payment_type|count|
+------------+-----+
| credit_card|76795|
|      boleto|19784|
|     voucher| 5775|
|  debit_card| 1529|
| not_defined|    3|
+------------+-----+



In [25]:
# Creating delivery_df with selected columns
delivery_df = orders_df.select(
    col("order_id"),
    col("order_purchase_timestamp"),
    col("order_delivered_customer_date")
)

In [34]:
from pyspark.sql.functions import datediff, col

# Creating a new DataFrame with average delivery time
average_delivery_df = delivery_df.withColumn(
    "average_delivery_time",
    datediff(col("order_delivered_customer_date"), col("order_purchase_timestamp"))
)

# Display the DataFrame
average_delivery_df.show()


+--------------------+------------------------+-----------------------------+---------------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|average_delivery_time|
+--------------------+------------------------+-----------------------------+---------------------+
|e481f51cbdc54678b...|     2017-10-02 10:56:33|          2017-10-10 21:25:13|                    8|
|53cdb2fc8bc7dce0b...|     2018-07-24 20:41:37|          2018-08-07 15:27:45|                   14|
|47770eb9100c2d0c4...|     2018-08-08 08:38:49|          2018-08-17 18:06:29|                    9|
|949d5b44dbf5de918...|     2017-11-18 19:28:06|          2017-12-02 00:28:42|                   14|
|ad21c59c0840e6cb8...|     2018-02-13 21:18:39|          2018-02-16 18:17:02|                    3|
|a4591c265e18cb1dc...|     2017-07-09 21:57:05|          2017-07-26 10:57:55|                   17|
|136cce7faa42fdb2c...|     2017-04-11 12:22:08|                         NULL|                 NULL|


In [35]:
average_delivery_df.orderBy("average_delivery_time", ascending=False).show()

+--------------------+------------------------+-----------------------------+---------------------+
|            order_id|order_purchase_timestamp|order_delivered_customer_date|average_delivery_time|
+--------------------+------------------------+-----------------------------+---------------------+
|ca07593549f1816d2...|     2017-02-21 23:31:27|          2017-09-19 14:36:39|                  210|
|1b3190b2dfa9d789e...|     2018-02-23 14:57:35|          2018-09-19 23:24:07|                  208|
|440d0d17af552815d...|     2017-03-07 23:59:51|          2017-09-19 15:12:50|                  196|
|2fb597c2f772eca01...|     2017-03-08 18:09:02|          2017-09-19 14:33:17|                  195|
|285ab9426d6982034...|     2017-03-08 22:47:40|          2017-09-19 14:00:04|                  195|
|0f4519c5f1c541dde...|     2017-03-09 13:26:57|          2017-09-19 14:38:21|                  194|
|47b40429ed8cce3ae...|     2018-01-03 09:44:01|          2018-07-13 20:51:31|                  191|
